# Preprocessing

In [1]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion Spectral Entropy

In [16]:
import librosa
import numpy as np
from scipy.stats import entropy

def calculate_spectral_entropy(audio_signal, sample_rate=16000, n_fft=1024, hop_length=512):
    """
    Berechnet die spektrale Entropie eines Audiosignals.

    Quelle:
        - Konzept von Entropy: Shannon Entropy (Informationsgehalt)
        - Librosa: Spektrale Leistungsdichte (PSD)
        - https://librosa.org/doc/main/generated/librosa.feature.spectral_flatness.html

    Args:
        audio_signal (np.array): Vorverarbeitetes Audio-Signal.
        sample_rate (int): Sampling-Rate des Audiosignals.
        n_fft (int): Anzahl der FFT-Punkte für die Spektralanalyse.
        hop_length (int): Schrittweite zwischen FFT-Berechnungen.

    Returns:
        float: Durchschnittliche spektrale Entropie des Signals oder 0 bei Stille.
    """
    # Berechnung des Leistungsdichtespektrums (PSD)
    psd = np.abs(librosa.stft(audio_signal, n_fft=n_fft, hop_length=hop_length))**2

    # Prüfe, ob das gesamte Signal still ist (keine Energie)
    total_energy = np.sum(psd)
    if total_energy < 1e-6:
        return 0.0  # Kein nennenswerter Informationsgehalt, daher Entropie = 0

    # Prüfe, ob es Frames mit Energie = 0 gibt
    frame_energy = np.sum(psd, axis=0)
    
    #if np.any(frame_energy == 0):
        #print(f"Warnung: Frames mit 0 Energie gefunden! Werte werden korrigiert. Anzahl: {np.sum(frame_energy == 0)}")

    # Falls ein Frame 0 Energie hat, setzen wir ihn auf einen kleinen Wert (1e-6)
    frame_energy[frame_energy == 0] = 1e-6

    # Normalisierung mit stabiler Mindestgrenze
    psd_norm = psd / frame_energy[None, :]

    # Prüfe auf NaN-Werte in der Normalisierung
    #if np.any(np.isnan(psd_norm)):
        #print("NaN-Warnung: NaN-Werte in der PSD-Normalisierung!")

    # Berechnung der Entropie pro Frame
    spectral_entropy = entropy(psd_norm, axis=0)

    # Falls nach der Berechnung noch NaN-Werte existieren, setze sie auf 0
    if np.any(np.isnan(spectral_entropy)):
        #print("NaN-Warnung: NaN-Werte in der Entropieberechnung! Setze auf 0.")
        spectral_entropy = np.nan_to_num(spectral_entropy, nan=0.0)

    return np.mean(spectral_entropy)

# Test

In [17]:
import os

# Verzeichnis mit den Audiodateien
audio_dir = "../audio_files/"

# Alle Audiodateien im Verzeichnis durchsuchen und alphabetisch sortieren
audio_files = sorted([f for f in os.listdir(audio_dir) if f.endswith(".wav")])

# Durch alle Audiodateien iterieren und Spectral Entropy berechnen
for file_name in audio_files:
    file_path = os.path.join(audio_dir, file_name)

    # Audio vorverarbeiten (Resampling, Mono, Normalisierung)
    audio_signal = preprocess_audio(file_path)

    # Spectral Entropy berechnen
    entropy_value = calculate_spectral_entropy(audio_signal)

    # Ergebnisse ausgeben
    print(f"Spectral Entropy ({file_name}): {entropy_value:.4f}")

Spectral Entropy (_noise_pink.wav): 4.0731
Spectral Entropy (_noise_white.wav): 5.8082
Spectral Entropy (_signal_constant.wav): 0.5429
Spectral Entropy (_signal_silence.wav): 0.0000
Spectral Entropy (_signal_sine.wav): 0.9222
Spectral Entropy (_snr_03.wav): 3.3357
Spectral Entropy (_snr_10.wav): 3.1912
Spectral Entropy (_snr_20.wav): 2.9255
Spectral Entropy (example1.wav): 2.3652
Spectral Entropy (example2.wav): 2.3383
Spectral Entropy (example3.wav): 2.6361
